In [ ]:
import requests
import pandas as pd
import json
from tqdm import tqdm


In [ ]:
class NAVERGeocoding:
    # https://api.ncloud-docs.com/docs/ai-naver-mapsgeocoding-geocode
    def __init__(self, secret_key_path: str, geocode_url: str) -> None:
        with open(secret_key_path, "r") as f:
            secret = json.load(f)
        self.__HEADER = {"X-NCP-APIGW-API-KEY-ID":secret["NAVER"]["ID"], "X-NCP-APIGW-API-KEY": secret["NAVER"]["SECRET_KEY"], "Accept":"application/json"}
        self.__GEOCODE_URL = geocode_url

    def _get_response(self, params: dict) -> dict:
        res = requests.get(url=self.__GEOCODE_URL,params=params, headers=self.__HEADER)
        assert res.status_code == int(200), res.status_code
        assert res.json()["status"] == "OK", res.json()["status"]
        return res.json()

    def get_normalized_address(self, address: str, count: int=1) -> tuple[str, str, str]:
        
        inp = {"query": address, "count": count}

        response = self._get_response(inp)
        list_responsed_address = response["addresses"]

        assert len(list_responsed_address) == count, print(response)
        responsed_address = list_responsed_address[0]
        
        NAVER_roadAddress = responsed_address["roadAddress"]
        x_coord = responsed_address["x"]
        y_coord = responsed_address["y"]
        return x_coord, y_coord,  NAVER_roadAddress
    
    def __call__(self, *args, **kwargs):
        return self.get_normalized_address(*args, **kwargs)


In [ ]:
geocoding = NAVERGeocoding(secret_key_path="/home/seungwon/workspace/massage/ts_code/sources/json/secret.json", geocode_url="https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode")

#load_path, save_path = "/home/seungwon/workspace/massage/locations/government/running_stores.csv", "government_geolocated.csv"
load_path, save_path = "/home/seungwon/workspace/massage/locations/anmawon/anmawon.csv", "anmawon_geolocated.csv"

data = pd.read_csv(load_path)
data.fillna("", inplace=True)
data["x"], data["y"], data["NAVER_addr"]= ["","",""]


In [ ]:
for idx, row in tqdm(data.iterrows()):
    try:
        addr = row["주소"]
        x, y, naver_addr = geocoding(addr)
        data.loc[idx, ["x", "y", "NAVER_addr"]] = x, y, naver_addr
    except Exception as e:  # 예외 발생한 아웃라이어에 대해서는 직접 수정 필요
        print(idx, e)
        pass


In [ ]:
data.to_csv(save_path,sep=",", header=True, index=False)